In [1]:
import psycopg2 as pg
import configparser

config = configparser.ConfigParser()
config.read('../bdtp4.config')

dbName = config['db']['name']
user = config['db']['user']
password = config['db']['password']

con = pg.connect(dbname=dbName, user=user, password=password)
cur = con.cursor()

a) Dado um produto, listar os 5 comentários mais úteis e com maior
avaliação, e os 5 comentários mais úteis e com menor avaliação

In [20]:
import pandas as pd

prod_id = 1

query = """
SELECT * FROM (
    (
        SELECT * FROM review WHERE prod_id = %s ORDER BY helpful DESC,
        rating DESC LIMIT 5
    )
    UNION ALL
    (
        SELECT * FROM review WHERE prod_id = %s ORDER BY helpful DESC,
        rating ASC LIMIT 5
    )
) R;
"""
query = cur.mogrify(query, (prod_id, prod_id))

try:
    cur.execute(query)
    result = cur.fetchall()
except Exception as e:
    print(e)
    con.reset()
    
pd.DataFrame(result, columns=('time', 'prod_id', 'customer_id', 'rating', 'votes', 'helpful'))

,time,prod_id,customer_id,rating,votes,helpful
0,2000-07-28,1,A2JW67OY8U6HHK,5,10,9
1,2003-12-14,1,A2VE83MZF98ITY,5,6,5
2,2000-07-28,1,A2JW67OY8U6HHK,5,10,9
3,2003-12-14,1,A2VE83MZF98ITY,5,6,5


b) Dado um produto, listar os produtos similares com maiores vendas do que ele

In [101]:
import pandas as pd

prod_id = 1

query = """
SELECT P.id, P.asin, P.title, P.group, P.sales_rank FROM (
        (
            SELECT p.id, p.sales_rank, ps.second_prod_id 
            FROM (
                        product as p 
                    JOIN 
                        product_similar as ps 
                    ON p.id = ps.first_prod_id
                 ) 
            WHERE p.id = %s
        ) as R
    JOIN
        product as P
    ON P.id = R.second_prod_id
) wHERE P.sales_rank < R.sales_rank
"""

query = cur.mogrify(query, (prod_id, ))

try:
    cur.execute(query)
    result = cur.fetchall()
except Exception as e:
    print(e)
    con.reset()
    
product_columns = ('id', 'ASIN', 'title', 'group', 'sales rank')
pd.DataFrame(result, columns=product_columns)

,id,ASIN,title,group,sales rank
0,118052,0687023955,The Four Pages of the Sermon: A Guide to Bibli...,Book,64877
1,161555,0804215715,Witness of Preaching,Book,93405
2,244916,156101074X,The Preaching Life,Book,92111
3,444232,0687074231,Performing the Word: Preaching As Theatre,Book,235453


c) Dado um produto, mostrar a evolução diária das médias de avaliação
ao longo do intervalo de tempo coberto no arquivo de entrada

In [104]:
import pandas as pd

prod_id = 1

query = """
SELECT *
FROM (
            product as P
        JOIN
            review as R
        ON P.id = R.prod_id
     )
WHERE id = %s
ORDER BY time;
"""

query = cur.mogrify(query, (prod_id, ))

try:
    cur.execute(query)
    result = cur.fetchall()
except Exception as e:
    print(e)
    con.reset()
    
pd.DataFrame(result)

,0,1,2,3,4,5,6,7,8,9,10
0,1,0827229534,Patterns of Preaching: A Sermon Sampler,Book,396585,2000-07-28,1,A2JW67OY8U6HHK,5,10,9
1,1,0827229534,Patterns of Preaching: A Sermon Sampler,Book,396585,2003-12-14,1,A2VE83MZF98ITY,5,6,5


d) Listar os 10 produtos lideres de venda em cada grupo de produtos

In [110]:
import pandas as pd

group = "Book"

query = """
SELECT *
FROM product
WHERE product.group = %s
ORDER BY sales_rank ASC LIMIT 10
"""

query = cur.mogrify(query, (group, ))

try:
    cur.execute(query)
    result = cur.fetchall()
except Exception as e:
    print(e)
    con.reset()
    
pd.DataFrame(result)

,0,1,2,3,4
0,11520,0970684711,The Ink Of Heaven,Book,-1
1,4543,319001504X,"Pingpong 1, coursebook",Book,-1
2,10890,156410026X,Up Close Metro Almanac Los Angles/Binder,Book,-1
3,10700,0076067661,"Ncarb Architectural Registration Handbook, 198...",Book,-1
4,1727,0780357213,Vehicular Technology 51st Conference Proceedings,Book,-1
5,7971,9201610866,"Gamma, X-Ray and Neutron Techniques for the Co...",Book,-1
6,2617,1880608014,DIMENSIONS OF FAITH,Book,-1
7,693,1853461822,Labour and Locality: Uneven Development and th...,Book,-1
8,2996,0467700036,Fracture Mechanics,Book,-1
9,14801,0945951043,Linking the Export Processing Zone to Local In...,Book,-1


e) Listar os 10 produtos com a maior media de avaliações uteis

In [113]:
import pandas as pd

query = """
SELECT id,title,AVG(helpful)
FROM product
JOIN review ON product.id = review.prod_id
GROUP BY (id,title)
ORDER BY AVG(helpful) DESC LIMIT 10
"""


try:
    cur.execute(query)
    result = cur.fetchall()
except Exception as e:
    print(e)
    con.reset()
    
pd.DataFrame(result, columns=('id', 'title', 'AVG(helpful)'))

,id,title,AVG(helpful)
0,380551,Understanding Loved Boys and Boylovers,243.0000000000000000
1,110544,T'ai Chi for Older Adults,233.0000000000000000
2,523037,More Than Just Hot Air: Common Sense Counter-T...,203.0000000000000000
3,178270,Crockpot Cookery (Cookbooks By Morris Press),197.0000000000000000
4,250701,Creative Interventions for Troubled Children &...,196.0000000000000000
5,312621,The Story About Ping,186.4651162790697674
6,411581,The Story About Ping (8x 8),186.3488372093023256
7,403662,"The Story about Ping : StoryTape (StoryTape, P...",186.3255813953488372
8,357193,The Story about Ping,186.3023255813953488
9,69111,"The Smoked-Foods Cookbook: How to Flavor, Cure...",183.0000000000000000
